In [1]:
import os 
import re
import string
from cleantext import clean
import requests
import json
import glob
import os
import pandas as pd
import numpy as np
import dateparser

In [3]:
#!pip install clean-text
#!pip install unidecode

In [4]:
# root
root = os.getcwd()
print(root)

# file
file = "data/order-vision-financial-markets-llc.txt"
print(file)

# text_path
text_path = root + "/" + file
print(text_path)

# sanitary check
os.path.isfile(text_path)

/home/alex/Desktop/legal_doc_processing/notebooks
data/order-vision-financial-markets-llc.txt
/home/alex/Desktop/legal_doc_processing/notebooks/data/order-vision-financial-markets-llc.txt


True

In [15]:
with open(text_path) as f:
    file = f.read()  

FileNotFoundError: [Errno 2] No such file or directory: '/home/alex/Desktop/legal_doc_processing/notebooks/order-vision-financial-markets-llc.txt'

In [4]:
def is_section_num(text):
    return re.search(r'^[MDCLXVIAB]+\.$',text) is not None
is_section_num("IV.")

True

In [6]:
punctuation = '!.?'

In [7]:
def ends_with_ponc(text):
    return text[-1] in punctuation if text else False 
ends_with_ponc("Propex engaged in sanctions.")

True

In [8]:
def start_with_upper(text):
    return re.match('^[A-Z]', text) is not None
start_with_upper("Fsfsd.")

True

In [9]:
def is_title(text):
    uppers = [word.isupper() for word in text.split(" ")]
    return sum(uppers) / len(uppers) > 0.6
is_title("UNITED STATES OF AMERICA Before the COMMODITY FUTURES TRADING COMMISSION")

True

In [10]:
import nltk
from nltk.tokenize import sent_tokenize
t = sent_tokenize("Three days later, the SEC  filed a federal injunctive action against Sentinel, SEC v.  Sentinel,  et al., No.  07 CV 4684 (N.D. Ill. filed Aug. 20, 2007), and on April 28, 2008, the CFTC filed a Complaint against Sentinel, Bloom and Sentinel's Senior Vice President and head trader, Charles K. Mosley, seeking injunctive and other equitable relief, as well as the imposition of civil penalties, for violating various provisions of the Act and Commission Regulations.")

In [11]:
def same_sentence(sent1, sent2):
    #print(sent1,"\n",sent2)
    
    # sent empty
    if (not sent1) or (not sent1['text']) :
        return True
    # section number
    if sent1['is_section_num']:
        return True
    
    # very short sentence
    if len(sent1['text']) < 50:
        return False
    
    # ponctuation
    if sent1['ends_with_ponc']:
        return False
    
    if sent1['is_title']:
        return False

    if sent2['is_title']:
        return False
    
    return True

In [12]:
def clean_doc(file_text):
    pages=[]
    for page in file_text.split("\x0c"):
        # clen text
        page_meta = [{'text': clean(para, lower=False, no_line_breaks=True).replace("_","")} 
                            for para in page.split('\n')]
        clean_page = []
        previous_line = {}
        text= ""
        # add meta data
        for line in page_meta:
            line['is_section_num'] = is_section_num(line['text'])
            line['is_title'] = is_title(line['text'])
            line['ends_with_ponc'] = ends_with_ponc(line['text'])
            line['is_alpha'] = sum(c.isalpha() for c in line['text'])
            line['start_with_upper']  = start_with_upper(line['text'])
                                
            # not relevant line
            if not line['is_alpha']:
                continue
            
            if not same_sentence(previous_line, line):
                if text:
                    clean_page.append(text)
                    text = ""
                
            previous_line = line
            text = " ".join([text, line['text']])
        if len(clean_page):
            pages.append(clean_page)
    return pages

## No case/docket

In [155]:
clean_page = clean_doc(file)
clean_page[1][2:4]

[' A. SUMMARY',
 ' From at least January 1 to November 30, 2014 (the "Relevant Period"), Respondent, a then-registered futures commission merchant ("FCM"), failed to diligently supervise the handling by its employees and agents of commodity interest accounts carried by Respondent and introduced by a Respondent guaranteed introducing broker ("GIB"), as well as the activities of its employees and agents relating to its business as a then-registered FCM to ensure compliance with the Act and Regulations, and to deter and detect wrongdoing, in violation of Regulation 166.3, 17 C.F.R. SS 166.3 (2018).']

In [14]:
def get_case(first_page):
    for line in first_page:
        if len(line) < 40:
            p = re.compile("NO[\.:]\s*.+")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()
            
    for line in first_page:   
        if len(line) < 40:    
            p = re.compile("\d*-?CV-\d+.*")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()

In [15]:
get_case(clean_page[0])

In [16]:
get_case([' CFTC Docket No. SD 20-01'])

'NO. SD 20-01'

## Defendant

In [38]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("\n".join(clean_page[0]))
doc = nlp(text)

# Analyze syntax
#print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
#print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    if entity.label_ == "ORG":
        print(entity.text,":       ", entity.label_)

CFTC Docket :        ORG
THE COMMODITY EXCHANGE ACT :        ORG
The Commodity Futures Trading Commission :        ORG
Vision Financial Markets LLC :        ORG
Commission Regulation :        ORG
Commission :        ORG
this Order Instituting Proceedings Pursuant to Section 6 :        ORG
the Commodity Exchange Act :        ORG
Imposing Remedial Sanctions :        ORG


In [34]:
print("\n".join(clean_page[0]))

 UNITED STATES OF AMERICA
 Before the
 COMMODITY FUTURES TRADING COMMISSION
 In the Matter of:
 Vision Financial Markets LLC,
 ) CFTC Docket No.
 Respondent.
 ORDER INSTITUTING PROCEEDINGS PURSUANT TO
 SECTIONS 6(c) AND 6(d) OF THE COMMODITY EXCHANGE ACT, MAKING
 FINDINGS AND IMPOSING REMEDIAL SANCTIONS
 I. INTRODUCTION
 The Commodity Futures Trading Commission ("Commission") has reason to believe that Vision Financial Markets LLC ( "Respondent") violated Commission Regulation ("Regulation") 166.3, 17 C.F.R. SS 166.3 (2018). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Respondent engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.
 In anticipation of the institution of an administrative proceeding, Respondent has submitted an Offer of Settlement ("Offer"), which the Commission has determined to ac

In [156]:
#!pip install transformers
#!pip3 install torch torchvision torchaudio
#!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

In [53]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Question answering pipeline, specifying the checkpoint identifier
nlp = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')

In [182]:
first_page = [text for text in clean_page[0] if len(text) > 100]

In [183]:
first_page

[' The Commodity Futures Trading Commission ("Commission") has reason to believe that Vision Financial Markets LLC ( "Respondent") violated Commission Regulation ("Regulation") 166.3, 17 C.F.R. SS 166.3 (2018). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Respondent engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.',
 ' In anticipation of the institution of an administrative proceeding, Respondent has submitted an Offer of Settlement ("Offer"), which the Commission has determined to accept.',
 ' Without admitting or denying any of the findings or conclusions herein, Respondent consents to the entry of this Order Instituting Proceedings Pursuant to Section 6( c) and ( d) of the Commodity Exchange Act, Making Findings, and Imposing Remedial Sanctions ("Order") and acknowledges service of this Orde

In [184]:
print(nlp(question="Who violeted?", context=".".join(first_page)))

{'score': 0.1251426488161087, 'start': 84, 'end': 112, 'answer': 'Vision Financial Markets LLC'}


In [185]:
print(nlp(question="Who is the defendant?", context=".".join(first_page), topk=3))

[{'score': 0.22406990826129913, 'start': 84, 'end': 112, 'answer': 'Vision Financial Markets LLC'}, {'score': 0.040718793869018555, 'start': 84, 'end': 128, 'answer': 'Vision Financial Markets LLC ( "Respondent")'}, {'score': 0.02145378664135933, 'start': 84, 'end': 108, 'answer': 'Vision Financial Markets'}]


# STOP HERE

## Extracted authorities

In [ ]:
UNITED STATES DISTRICT COURT
FOR THE MIDDLE DISTRICT OF FLORIDA 

## Code law violation

In [ ]:
violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C.
§ 6c(a)(5)(C) (2018).

In [ ]:
Section 4c(a)(5) of the Act, 7 U.S.C.
§ 6c(a)(5) (2012).

## Violation period

In [428]:
line = 'The Commodity Futures Trading Commission ("Commission") has reason to believe that from at least July 2012 through March 2017 ("Relevant Period"), Propex Derivatives Pty Ltd. ("Propex") violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C. 6c(a)(5)(C) (2018). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Propex engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.'

In [ ]:
("Relevant Period")

## Transaction amount

In [ ]:
pour complaint tous les montants vers Transaction amounts

## Defendant

In [ ]:
Propex Derivatives Pty Ltd, Respondent.

In [18]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm

## Nature of violations

In [ ]:
II. FINDINGS
The Commission finds the following:
A. SUMMARY
During the Relevant Period, Propex, by and through a Propex trader ("Trader A"),·
engaged in thousands of instances of the disruptive trading practice known as "spoofing"
(bidding or offering with the intent to cancel the bid or offer before execution) in the E-mini
S&P 500 futures contracts traded on the Chicago Mercantile Exchange ("CME"), a futures
exchange and designated contract market which is owned and operated by CME Group Inc. This
conduct violated Section 4c(a)(5)(C) of the Act, 7 U.S.C. § 6c(a)(5)(C) (2018). 

In [66]:
cftc_full_list = pd.read_excel("cftc_full_list.xlsx")

In [68]:
def gen_line_rep(rep):
    original_path = f"./cftc/original/{rep}"
    text_path = f"./cftc/text/{rep}"
    meta_path = f"./cftc/meta-data/{rep}"
    lines = []
    for file in glob.glob(f"{meta_path}/*.json"):
        filename = os.path.basename(file)

        with open(file) as json_file: 
            data = json.load(json_file)     

        data['filename'] = filename.replace(".json","")
        data['folder'] = rep
        with open(f"{text_path}/{filename.replace('.json','.txt')}") as f:
            data['doc_text'] = f.read()  


        lines.append(data)
    return lines

In [69]:
meta_data = []
for index, row in cftc_full_list[~cftc_full_list.scraped_folder.isnull()].iterrows():
    try:
        meta_data.extend(gen_line_rep(row['scraped_folder']))
    except:
        print("error on",  row['scraped_folder'])
len(meta_data)

error on 7882-19
error on 8262-20
error on 8239-20


756

In [70]:
df_meta_data = pd.DataFrame(meta_data)

In [71]:
df_meta_data['doc_clean'] = df_meta_data.doc_text.apply(clean_doc)

In [72]:
df_meta_data['first_page'] = df_meta_data['doc_clean'].str[0]

In [73]:
df_meta_data = df_meta_data[~df_meta_data['first_page'].isnull()]

## Get reference

In [27]:
df_meta_data['reference'] = df_meta_data.first_page.apply(get_case)

## Get defendant

In [178]:
first_page = [text for text in df_meta_data.first_page.values[3] if len(text) > 100]

In [179]:
first_page

[' The Commodity Futures Trading Commission ("Commission") has reason to believe that from in or about August 2012 to at least January 2015 (the "Relevant Period"), Respondents Sharpe Signa, LLC ("Sharpe") and Garen Ovsepyan ("Ovsepyan") violated Sections 2(c)(2)(C)(iii)(I)(bb), 4b(a)(2)(A) and (C), 4o(l)(A) and (B), and 6(c)(2) of the Act, 7 U.S.C.',
 ' SSSS 2(c)(2)(C)(iii)(I)(bb), 6b(a)(2)(A) and (C), 6o(l)(A) and (B), and 9(2) (2012), and Commission Regulation ("Regulation") 5.3(a)(3)(i), 17 C.F.R. SS 5.3(a)(3)(i) (2014), and Respondent Haeres Capital, LLC ("Haeres") and Ovsepyan violated Section 6(c )(2) of the Act, 7 U.S.C. SS 9(2) (2012). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Respondent(s) engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.',
 ' In anticipation of the institution of a

In [180]:
print(nlp(question="Who violeted?", context=".".join(first_page)))

{'score': 0.1322660595178604, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}


In [181]:
print(nlp(question="Who is the defendant?", context=".".join(first_page), topk=3))

[{'score': 0.009618670679628849, 'start': 208, 'end': 222, 'answer': 'Garen Ovsepyan'}, {'score': 0.0038621597923338413, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}, {'score': 0.0027537851128727198, 'start': 175, 'end': 203, 'answer': 'Sharpe Signa, LLC ("Sharpe")'}]


In [141]:
#df_meta_data[['reference', 'folder', 'filename']][df_meta_data.reference.isnull()]

## Type

In [218]:
df_meta_data['is_order'] = df_meta_data.filename.str.contains("order")

In [219]:
df_meta_data['is_complaint'] = df_meta_data.filename.str.contains("complaint")

In [220]:
df_meta_data['type'] = np.where(df_meta_data['is_order'], "Order CFTC", 
         np.where(df_meta_data['is_complaint'], 
                  "Complaint CFTC", None))